In [ ]:
from minio.error import S3Error
import fitz  # PyMuPDF
from datetime import datetime
from io import BytesIO
import pandas as pd
import re
import psycopg2
import import_ipynb
from Utils import conectar_minio,transformar_tipo_identificacion, generar_ruta_fecha,extraer_archivo_minio, insert_into_info_lab, agregar_fuente, agregar_observacion, cargar_dfpaciente
import os
from dotenv import load_dotenv


In [ ]:
# Cargar variables del archivo .env
load_dotenv()

In [ ]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

In [ ]:
ruta_fecha = generar_ruta_fecha() 

In [ ]:
 #3. Extraer un archivo Excel/CSV/Parquet
df_datos = extraer_archivo_minio(
    minio_client=minio_client,
    bucket_name="yachay-bronze",
    ruta_archivo=f"laboratorios/{ruta_fecha}/laboratorio synlab/synlab.csv",
    tipo_archivo="csv"
)
print(df_datos)  # Mostrar las primeras filas del DataFrame extraído

In [ ]:
def transformar_df_lab_synlab(df_datos):
    """
    Transforma un DataFrame del laboratorio Jimenes a la estructura de id_infolab
    
    Parámetros:
    - df_datos: DataFrame con los campos originales del laboratorio
    
    Retorna:
    - DataFrame transformado con la estructura deseada
    """
    
    # Crear copia para no modificar el original
    #df_transformado = df_datos.copy()
    
    # Mapeo de campos
    mapeo_campos = {
        #infolab
        'fecha_toma':'fecha_toma_muestra',
        'fecha_ingr' :'fecha_ingreso',
        'fec_val': 'fecha_informe',
        'prestador': 'entidad',
        'descr_nit' : 'eps',
        'descr_examen' : 'servicio',
        'caracteristicas' : 'muestra_remitida',
        'desc_macro': 'descripcion_macroscopica',
        'desc_micro': 'descripcion_microscopica',
        'diagnostico': 'diagnostico',
        'observaciones2': 'comentario',
        'edad': 'Edad',
        'genero':'sexo',
        #
        #'': 'archivo',
        #'': 'cie10',
        #'': 'diagnostico_clinico',
        #'': 'medico_ordenante',
        #'': 'medico_remitente',        
        #paciente
        'tipo_documento' : 'TipoIdentificacion', #tipo_de_identificacion, se transformará el nombre de la col. en la función transformar_tipo_identificacion
        'documento' : 'NumeroIdentificacion', #identificacion, se transformará el nombre de la col. en la función transformar_tipo_identificacion
        'municipio' : 'municipio_residencia',
        #'': 'nombre1',
    }

    # Renombrar columnas
    df_transformado = df_datos.rename(columns=mapeo_campos)

    #se debe tratar los campos de edad y fechas
    df_transformado['Edad'] = (df_transformado['Edad'].astype(str).str.upper().str.replace('A', '', regex=False)
        .apply(lambda x: x if x.isdigit() else '0').astype(int))
    df_transformado['fecha_toma_muestra'] = (pd.to_datetime(
        df_transformado['fecha_toma_muestra'], errors='coerce', dayfirst=True)
        .dt.strftime('%Y-%m-%d').fillna(pd.NA).astype('string'))
    df_transformado['fecha_ingreso'] = (pd.to_datetime(
        df_transformado['fecha_ingreso'], errors='coerce', dayfirst=True)
        .dt.strftime('%Y-%m-%d').fillna(pd.NA).astype('string'))
    df_transformado['fecha_informe'] = (pd.to_datetime(
        df_transformado['fecha_informe'], errors='coerce', dayfirst=True)
        .dt.strftime('%Y-%m-%d').fillna(pd.NA).astype('string'))
    df_transformado['Nombre del Paciente'] =  (df_transformado[['nombre1', 'nombre2', 'apellido1', 'apellido2']].astype(str)
        .apply(lambda x: ' '.join(v for v in x if v and v.strip() and v.lower() != 'nan'), axis=1))

    return df_transformado

In [ ]:
df = transformar_df_lab_synlab(df_datos)
#print(df.columns)  # Mostrar las columnas del DataFrame transformado
df = transformar_tipo_identificacion(df)
df = agregar_fuente(df, "056")
df = agregar_observacion(df)
print(df.columns)  # Mostrar las columnas del DataFrame transformado

In [ ]:
connection_params = {
    'dbname': os.getenv("DB_NAME"),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'host': os.getenv("DB_HOST"),
    'port':os.getenv("DB_PORT")
}
insert_into_info_lab(df, connection_params)

In [ ]:
df_paciente = df[['NumeroIdentificacion', 'Nombre del Paciente', 'sexo', 'Edad', 'telefono', 'id_paciente','TipoIdentificacion']].copy()
df_paciente = df_paciente.drop_duplicates(subset=['id_paciente'], keep='first')
df_paciente['NumeroIdentificacion'] = df_paciente['NumeroIdentificacion'].astype(str).str.replace('.0', '', regex=False)
print(df_paciente.head())  # Mostrar las columnas del DataFrame transformado

In [ ]:
def renombrar_campos_paciente(df_paciente):
    """
    Renombra los campos del DataFrame df_paciente a los nombres especificados.

    Parámetros:
    - df_paciente: DataFrame con los campos originales.

    Retorna:
    - DataFrame con los campos renombrados.
    """
    mapeo_campos = {
        'Nombre del Paciente': 'nombre1',  # Asumiendo que este campo contiene el primer nombre
        'NumeroIdentificacion': 'identificacion',
        'sexo': 'sexo',
        'Edad': 'edad',
        'TipoIdentificacion': 'tipo_identificacion',    
        'id_paciente': 'id_paciente',
        # Agregar campos adicionales si están disponibles en el DataFrame original
    }
    
    # Renombrar columnas
    df_paciente_renombrado = df_paciente.rename(columns=mapeo_campos)
    
    # Agregar columnas faltantes con valores nulos
    columnas_faltantes = [
        'nombre2', 'nombre3', 'apellido1', 'apellido2',
        'fecha_nacimiento', 'municipio_residencia', 'codigo_residencia',
        'barrio_residencia', 'direccion'
    ]
    for columna in columnas_faltantes:
        if columna not in df_paciente_renombrado.columns:
            df_paciente_renombrado[columna] = None
    
    return df_paciente_renombrado

In [ ]:
df_paciente_renombrado = renombrar_campos_paciente(df_paciente)
print(df_paciente_renombrado.head())

In [ ]:
cargar_dfpaciente(df_paciente_renombrado,'paciente', connection_params)